In [46]:
import tensorflow as tf
import numpy as np

In [ ]:
Batch_Size = 3

filename_queue = tf.train.string_input_producer(["heart.csv"])
reader = tf.TextLineReader(skip_header_lines=1)
key,value = reader.read(filename_queue)
record_defaults = [[1.0] for i in range(9)]
record_defaults[4] = ['']
record_defaults.append([1])
content = tf.decode_csv(value,record_defaults=record_defaults)
condition = tf.equal(content[4],tf.constant('Present'))
content[4] = tf.select(condition,tf.constant(1.0),tf.constant(0.0))
features = tf.pack(content[:9])
label = content[-1]


minimum_after_dequeue = 10*Batch_Size
#capacity: Recommendation:   min_after_dequeue + 
#(num_threads + a small safety margin) * batch_size
capacity = 1.5*minimum_after_dequeue
data_batch,label_batch = tf.train.shuffle_batch([features,label],batch_size=Batch_Size+1,
                                               capacity=capacity,min_after_dequeue=minimum_after_dequeue)


final_features = []
final_labels = []

with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    #print sess.run(key)
    #data_batch,label_batch = sess.run([data_batch,label_batch])
    for _ in range(10):
        features,labels = sess.run([data_batch,label_batch])
        final_features.append(features)
        final_labels.append(labels)
        #print features,labels
    coord.request_stop()
    coord.join(threads)
print final_features,final_labels

In [47]:
Batch_Size = 10
feature_size = 9
def get_feature_label(filenames_list):
    filename_queue = tf.train.string_input_producer(filenames_list)
    reader = tf.TextLineReader(skip_header_lines=1)
    key,value = reader.read(filename_queue)
    record_defaults = [[1.0] for i in range(9)]
    record_defaults[4] = ['']
    record_defaults.append([1.0])
    content = tf.decode_csv(value,record_defaults=record_defaults)
    condition = tf.equal(content[4],tf.constant('Present'))
    content[4] = tf.select(condition,tf.constant(1.0),tf.constant(0.0))
    features = tf.pack(content[:feature_size])
    label = content[-1]


    minimum_after_dequeue = 10*Batch_Size
    #capacity: Recommendation:   min_after_dequeue + 
    #(num_threads + a small safety margin) * batch_size
    capacity = 1.5*minimum_after_dequeue
    data_batch,label_batch = tf.train.shuffle_batch([features,label],batch_size=Batch_Size,
                                                   capacity=capacity,min_after_dequeue=minimum_after_dequeue)
    label_batch = tf.reshape(label_batch,[Batch_Size,1])
    return data_batch,label_batch


In [48]:
X = tf.placeholder(tf.float32,[Batch_Size,feature_size],name='X_placeholder')
Y = tf.placeholder(tf.float32,[Batch_Size,1],name='Y_placeholder')
# w = tf.Variable([[0.1] for _ in range(9)],name="Weights")
w = tf.Variable(tf.random_normal([feature_size,1]),name="Weights")
b = tf.Variable(0.1,name="biases")
Y_predicted = tf.matmul(X,w) + b
squared_error = tf.square(Y-Y_predicted,name='squared_error')
loss = tf.reduce_mean(squared_error,name="loss")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=5e-6).minimize(loss)


In [49]:
data_batch,label_batch = get_feature_label(["heart.csv"])

In [56]:
final_features = []
final_labels = []
with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    #print sess.run(key)
    #data_batch,label_batch = sess.run([data_batch,label_batch])
    for _ in range(30):
        features,labels = sess.run([data_batch,label_batch])
        final_features.append(features)
        final_labels.append(labels)
    coord.request_stop()
    coord.join(threads)
    
print len(final_features),len(final_labels)    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(400):
        total_loss = 0
        for i in range(len(final_features)):
            if abs(i-len(final_features)) > 2: 
                temp_labels = [j[0] for j in final_labels[i]]
                temp_labels = np.array(temp_labels)
                temp_labels = np.reshape(temp_labels,(-1,1))
                temp_features = np.array(final_features[i])
                _,l = sess.run([optimizer,loss],feed_dict={X:temp_features,Y:temp_labels})
                total_loss+=l
            else:
                temp_labels = [j[0] for j in final_labels[i]]
                temp_labels = np.array(temp_labels)
                temp_labels = np.reshape(temp_labels,(-1,1))
                X_test = np.array(final_features[i])
                _,l,prediction = sess.run([optimizer,loss,Y_predicted],feed_dict={X:X_test,Y:temp_labels})
                print prediction,labels
        #print 'Epoch {0}: {1}'.format(j, total_loss/(len(final_features)))

30 30
[[-61.3092041 ]
 [-25.10798073]
 [ 44.55934906]
 [ -4.95766592]
 [ -0.76873726]
 [ 45.17620087]
 [-63.95822144]
 [ 39.18119812]
 [-20.44492722]
 [-21.57501984]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 13.17616653]
 [-52.73839951]
 [-58.12216187]
 [ 27.15494537]
 [-70.02128601]
 [ 27.07441139]
 [-28.02989388]
 [ 19.71945953]
 [ -1.3249929 ]
 [ 16.86283875]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[-53.79314423]
 [-21.44219971]
 [ 40.1251564 ]
 [ -3.27766967]
 [ -2.06183028]
 [ 40.42411804]
 [-58.35909271]
 [ 35.33852768]
 [-20.72778893]
 [-22.66731071]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 11.22675991]
 [-47.34285736]
 [-49.14413452]
 [ 24.4918251 ]
 [-57.67469788]
 [ 24.61765099]
 [-24.34167862]
 [ 16.83928299]
 [ -1.11388111]
 [ 15.45531368]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[-47.19775772]
 [-18.26026726]
 [ 36.24141312]
 [ -1.85299122]
 [

[[ -1.54976547]
 [ -0.47338569]
 [  4.77993298]
 [  2.21196556]
 [ -3.77513218]
 [  0.93103611]
 [-10.02094746]
 [  1.08837521]
 [-14.92372608]
 [-16.05937958]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[  1.52163875e+00]
 [ -9.26712418e+00]
 [  3.37370110e+00]
 [  6.98528099e+00]
 [  1.04477987e+01]
 [  3.37814713e+00]
 [ -7.04202801e-03]
 [ -3.70201683e+00]
 [ -1.52899754e+00]
 [  6.38605118e+00]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ -1.28795576]
 [ -0.48000669]
 [  4.50082254]
 [  2.09836912]
 [ -3.63902617]
 [  0.6058917 ]
 [ -9.53898048]
 [  0.70357943]
 [-14.67711258]
 [-15.6900177 ]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[  1.52659881]
 [ -9.02545834]
 [  3.40524936]
 [  6.87727642]
 [ 10.3690052 ]
 [  3.18685889]
 [  0.07199942]
 [ -3.80759645]
 [ -1.55926716]
 [  6.31060886]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ -1.04921365]
 [ -0.4930985 ]


[[ 1.46916413]
 [-1.33822751]
 [ 1.32982564]
 [ 0.25041887]
 [-1.61209083]
 [-2.88042378]
 [-2.86580348]
 [-4.87544346]
 [-9.89604568]
 [-9.7632246 ]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 1.53411674]
 [-5.40070248]
 [ 2.47843552]
 [ 4.98478127]
 [ 6.92358589]
 [ 1.00640202]
 [ 0.60371113]
 [-4.39235306]
 [-1.67979002]
 [ 4.73129559]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 1.51340878]
 [-1.37392843]
 [ 1.28368866]
 [ 0.21103251]
 [-1.5708636 ]
 [-2.92130423]
 [-2.72219658]
 [-5.00159836]
 [-9.74706936]
 [-9.6238575 ]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 1.52840006]
 [-5.29853678]
 [ 2.44097877]
 [ 4.92318678]
 [ 6.80826283]
 [ 0.97480315]
 [ 0.60720795]
 [-4.37326193]
 [-1.66882169]
 [ 4.67703962]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 1.55546772]
 [-1.40921056]
 [ 1.24077523]
 [ 0.17349546]
 [-1.53150928]
 [-2.95835042]
 [-2.58411765]
 [-5.12

[[ 2.15240812]
 [-2.14306974]
 [ 0.82988942]
 [-0.34707728]
 [-0.94663751]
 [-3.13670158]
 [-0.42676392]
 [-7.05743504]
 [-6.64943171]
 [-7.33481646]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 1.39657402]
 [-3.15991688]
 [ 1.86147118]
 [ 3.5914917 ]
 [ 4.69204569]
 [ 0.63874197]
 [ 0.69266129]
 [-3.55589485]
 [-1.28583908]
 [ 3.55776882]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 2.16652322]
 [-2.16959786]
 [ 0.82864934]
 [-0.35836095]
 [-0.93137234]
 [-3.12687206]
 [-0.37013024]
 [-7.10883427]
 [-6.54562044]
 [-7.2757163 ]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 1.39327776]
 [-3.08765221]
 [ 1.84882414]
 [ 3.5460906 ]
 [ 4.6303854 ]
 [ 0.63571793]
 [ 0.69695526]
 [-3.51616812]
 [-1.26903427]
 [ 3.52203131]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 2.1801095 ]
 [-2.19584751]
 [ 0.82813829]
 [-0.36912209]
 [-0.91652352]
 [-3.11632133]
 [-0.31530434]
 [-7.15

[[ 1.36719513]
 [-1.53026342]
 [ 1.65852499]
 [ 2.57973051]
 [ 3.42033267]
 [ 0.64550179]
 [ 0.81915659]
 [-2.51899052]
 [-0.87378198]
 [ 2.80756474]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 2.403579  ]
 [-2.76871443]
 [ 0.96079952]
 [-0.52406806]
 [-0.63186234]
 [-2.7224164 ]
 [ 0.6301797 ]
 [-8.00244808]
 [-4.23885345]
 [-6.16476965]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 1.36798429]
 [-1.47681808]
 [ 1.65417671]
 [ 2.54714966]
 [ 3.38220596]
 [ 0.64770323]
 [ 0.82425648]
 [-2.48079348]
 [-0.85942811]
 [ 2.78506136]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 2.40945792]
 [-2.78852439]
 [ 0.96932054]
 [-0.52727222]
 [-0.6226685 ]
 [-2.70432591]
 [ 0.65582919]
 [-8.02432823]
 [-4.1633296 ]
 [-6.13346338]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 1.36887491]
 [-1.42398226]
 [ 1.64997756]
 [ 2.51498318]
 [ 3.34469652]
 [ 0.64996302]
 [ 0.82934892]
 [-2.44

...

[[ 2.85779047]
 [-3.98102522]
 [ 1.92974651]
 [-0.54257822]
 [ 0.30330989]
 [-0.74963611]
 [ 1.8894825 ]
 [-8.14695168]
 [ 1.43752789]
 [-3.62615538]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 1.66408229]
 [ 2.53551006]
 [ 1.19997156]
 [ 0.27655295]
 [ 0.9919126 ]
 [ 0.87503123]
 [ 1.12366748]
 [ 0.58123618]
 [ 0.22700426]
 [ 1.45664537]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 2.8591547 ]
 [-3.98052192]
 [ 1.93128943]
 [-0.54211134]
 [ 0.30626717]
 [-0.74440289]
 [ 1.89070332]
 [-8.14056587]
 [ 1.44649327]
 [-3.61812472]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 1.66457355]
 [ 2.54197001]
 [ 1.19726431]
 [ 0.27334905]
 [ 0.98917001]
 [ 0.8750211 ]
 [ 1.12300241]
 [ 0.58551514]
 [ 0.22857362]
 [ 1.45504379]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 2.8605063 ]
 [-3.97997975]
 [ 1.93280578]
 [-0.54164314]
 [ 0.3092072 ]
 [-0.73920578]
 [ 1.89190209]
 [-8.13

[[ 2.88962674]
 [-3.9569633 ]
 [ 1.96197426]
 [-0.53037536]
 [ 0.37484214]
 [-0.62464458]
 [ 1.91466486]
 [-7.97460842]
 [ 1.63590682]
 [-3.42751431]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 1.67352021]
 [ 2.67799282]
 [ 1.13085759]
 [ 0.20683357]
 [ 0.93409836]
 [ 0.87244761]
 [ 1.10392737]
 [ 0.67325246]
 [ 0.26088995]
 [ 1.42138278]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 2.89070082]
 [-3.95562696]
 [ 1.96291244]
 [-0.5299046 ]
 [ 0.37738305]
 [-0.62026119]
 [ 1.91538393]
 [-7.96775436]
 [ 1.6422292 ]
 [-3.42027307]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 1.67375517]
 [ 2.68250656]
 [ 1.12826085]
 [ 0.20466045]
 [ 0.93237114]
 [ 0.87226021]
 [ 1.10308373]
 [ 0.67607665]
 [ 0.26193309]
 [ 1.42025959]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 2.89176297]
 [-3.9542625 ]
 [ 1.96383119]
 [-0.52943414]
 [ 0.3799082 ]
 [-0.61590701]
 [ 1.91608739]
 [-7.96

[[ 2.91399002]
 [-3.91424799]
 [ 1.98027098]
 [-0.51817656]
 [ 0.43630004]
 [-0.51935965]
 [ 1.9284004 ]
 [-7.7925539 ]
 [ 1.77448595]
 [-3.24777651]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 1.67701113]
 [ 2.77598739]
 [ 1.0651511 ]
 [ 0.16027869]
 [ 0.89860392]
 [ 0.8658703 ]
 [ 1.08077502]
 [ 0.73303443]
 [ 0.28288329]
 [ 1.3964628 ]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 2.91478062]
 [-3.91230893]
 [ 1.98074102]
 [-0.51770955]
 [ 0.43848255]
 [-0.51564354]
 [ 1.92873979]
 [-7.78542137]
 [ 1.77884436]
 [-3.2412014 ]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 1.67704451]
 [ 2.77902198]
 [ 1.06270409]
 [ 0.15885915]
 [ 0.89758354]
 [ 0.86555248]
 [ 1.07984531]
 [ 0.73482758]
 [ 0.28353649]
 [ 1.39565861]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 2.91556191]
 [-3.91034961]
 [ 1.98119712]
 [-0.51724178]
 [ 0.44065189]
 [-0.51194954]
 [ 1.9290669 ]
 [-7.77

 [ 1.37961721]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 2.93063021]
 [-3.86046648]
 [ 1.98767257]
 [-0.5065667 ]
 [ 0.48720157]
 [-0.4328025 ]
 [ 1.9333694 ]
 [-7.61227751]
 [ 1.86568904]
 [-3.09009385]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 1.67596292]
 [ 2.83819318]
 [ 1.00593877]
 [ 0.13153905]
 [ 0.87909287]
 [ 0.85676718]
 [ 1.05718184]
 [ 0.76897907]
 [ 0.29572561]
 [ 1.37901187]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 2.93116307]
 [-3.85810852]
 [ 1.98779452]
 [-0.50610578]
 [ 0.48908639]
 [-0.42959499]
 [ 1.93342817]
 [-7.60500145]
 [ 1.86863792]
 [-3.08406258]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 1.67584407]
 [ 2.84013867]
 [ 1.00365746]
 [ 0.13065194]
 [ 0.87854147]
 [ 0.85635763]
 [ 1.056229  ]
 [ 0.77007258]
 [ 0.29610157]
 [ 1.37841189]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 2.93168569]
 [-3.85573

[[ 2.9412775 ]
 [-3.79508686]
 [ 1.9871285 ]
 [-0.49468207]
 [ 0.53277564]
 [-0.35496026]
 [ 1.93194926]
 [-7.42226362]
 [ 1.92792571]
 [-2.9396019 ]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 1.67127931]
 [ 2.87738442]
 [ 0.94889235]
 [ 0.11384837]
 [ 0.86916256]
 [ 0.84518456]
 [ 1.03258717]
 [ 0.79067689]
 [ 0.30270427]
 [ 1.36525536]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 2.9415586 ]
 [-3.79242134]
 [ 1.98696685]
 [-0.4942289 ]
 [ 0.53439218]
 [-0.35217917]
 [ 1.93178391]
 [-7.4149375 ]
 [ 1.92977762]
 [-2.93406177]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 1.67103803]
 [ 2.87846041]
 [ 0.94679344]
 [ 0.11336776]
 [ 0.86894089]
 [ 0.84470487]
 [ 1.03165376]
 [ 0.79126346]
 [ 0.30286831]
 [ 1.36479044]] [[ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
[[ 2.94182968]
 [-3.78974676]
 [ 1.98679471]
 [-0.49377659]
 [ 0.53599954]
 [-0.34941393]
 [ 1.93160939]
 [-7.40